In [1]:
# Prerequisites
import json
import os
from eye_ai.eye_ai import EyeAI
from deriva_ml.dataset_bag import DatasetBag

import pandas as pd
from pathlib import Path, PurePath
import logging

from deriva.core.ermrest_model import Model, Schema, Table, Column, Key, ForeignKey, builtin_types

from deriva_ml import DatasetSpec, ExecutionConfiguration, DerivaML, Workflow, VersionPart
from deriva_ml import MLVocab as vc
from deriva_ml.deriva_definitions import ColumnDefinition, BuiltinTypes
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', force=True)

2025-04-09 17:02:11.581894: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-09 17:02:11.581952: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-09 17:02:11.591467: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-09 17:02:11.616838: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-09 17:02:14.418179: W tensorflow/compiler/tf2

In [2]:
from deriva.core.utils.globus_auth_utils import GlobusNativeLogin

# host = 'www.eye-ai.org'
host = 'dev.eye-ai.org'
catalog_id = "eye-ai"

gnl = GlobusNativeLogin(host=host)
if gnl.is_logged_in([host]):
    print("You are already logged in.")
else:
    gnl.login([host], no_local_server=True, no_browser=True, refresh_tokens=True, update_bdbag_keychain=True)
    print("Login Successful")

You are already logged in.


In [3]:
cache_dir = '/data'
working_dir = '/data'
EA = EyeAI(hostname = host, catalog_id = catalog_id, cache_dir= cache_dir, working_dir=working_dir)

2025-04-09 17:02:17,502 - WARNING - nbstripout is not installed in repository. Please run nbstripout --install


In [4]:
c_domain = EA.catalog.getPathBuilder().schemas['eye-ai']
c_ml = EA.catalog.getPathBuilder().schemas['deriva-ml']

m_domain = EA.model.schemas['eye-ai']
m_ml = EA.model.schemas['deriva-ml']

exec_asset_c = c_ml.tables['Execution_Asset']
exec_meta_c = c_ml.tables['Execution_Metadata']
exec_c = c_ml.tables['Execution']
exec_asset_type_c = c_ml.tables['Execution_Asset_Type']
exec_meta_type_c = c_ml.tables['Execution_Metadata_Type']


exec_asset_m = m_ml.tables['Execution_Asset']
exec_meta_m = m_ml.tables['Execution_Metadata']
exec_m = m_ml.tables['Execution']

In [6]:
def create_asso(table1, table2, schema):
    asso_def = Table.define_association([table1, table2])
    return schema.create_table(asso_def)
    
def create_asset_type():
    table_def_asset_type_vocab = Table.define_vocabulary(
        tname='Asset_Type', curie_template='eye-ai:{RID}'
    )

    asset_type_table = m_ml.create_table(table_def_asset_type_vocab)
    # asset_type_table = m_ml.tables['Asset_Type']

    for t in [exec_asset_m, exec_meta_m]:
        create_asso(t, asset_type_table, m_ml)

In [ ]:
# create_asset_type()

In [ ]:
def ingest_asset_type():
    asset_type_table = c_ml.tables['Asset_Type']
    cols = ["Name", "Description"]
    asset_type = [{c: item[c] for c in cols} for item in exec_asset_type_c.entities()]
    meta_type = [{c: item[c] for c in cols} for item in exec_meta_type_c.entities()]
    asset_type_table.insert(asset_type, defaults={'URI', 'ID'})
    asset_type_table.insert(meta_type, defaults={'URI', 'ID'})

def ingest_asset_type_asso(old_asset_t, old_asset_name, asso_t, old_type_name):
    entities = [{old_asset_name: item['RID'], "Asset_Type": item[old_type_name]} for item in old_asset_t.entities()]
    asso_t.insert(entities)
    

In [ ]:
# ingest_asset_type()
# ingest_asset_type_asso(exec_asset_c, 'Execution_Asset', c_ml.tables['Execution_Asset_Asset_Type'], 'Execution_Asset_Type')
# ingest_asset_type_asso(exec_meta_c, 'Execution_Metadata', c_ml.tables['Execution_Metadata_Asset_Type'], 'Execution_Metadata_Type')


In [ ]:
def create_asset_role():
    table_def_role_vocab = Table.define_vocabulary(
        tname='Asset_Role', curie_template='eye-ai:{RID}'
    )
    # asset_type_table = m_ml.create_table(table_def_role_vocab)
    asset_type_table = m_ml.tables['Asset_Role']
    
    ass_asso_table = m_ml.tables['Execution_Asset_Execution']
    meta_asso_table = m_ml.tables['Execution_Metadata_Execution']
    ass_asso_table.create_column(Column.define('Asset_Role', builtin_types.text, nullok=True))
    meta_asso_table.create_column(Column.define('Asset_Role', builtin_types.text, nullok=True))

    ass_asso_table.create_fkey(
        ForeignKey.define(
        ['Asset_Role'],
        'deriva-ml', 'Asset_Type',
        ['Name']
    ))
    meta_asso_table.create_fkey(
        ForeignKey.define(
        ['Asset_Role'],
        'deriva-ml', 'Asset_Type',
        ['Name']
    ))


In [ ]:
# create_asset_role()

In [8]:
fbx_table = m_domain.tables['Fundus_Bounding_Box']
exec_m = m_ml.tables['Execution']
asset_type_m = m_ml.tables['Asset_Type']
# asso = create_asso(fbx_table, exec_m, m_domain)
# asso = create_asso(fbx_table, asset_type_m, m_domain)